Notebook for testing docking against Mpro

In [1]:
from ccdc.docking import Docker
from ccdc.io import MoleculeReader, EntryReader

docker = Docker()
settings = docker.settings

MLL1_protein_file = '/Applications/CCDC/Python_API_2022/docs/example_files/2w5y_protein_prepared.mol2'
settings.add_protein_file(MLL1_protein_file)

MLL1_native_ligand_file = '/Applications/CCDC/Python_API_2022/docs/example_files/SAH_native.mol2'
MLL1_native_ligand = MoleculeReader(MLL1_native_ligand_file)[0]
MLL1_protein = settings.proteins[0]
settings.binding_site = settings.BindingSiteFromLigand(MLL1_protein, MLL1_native_ligand, 8.0)

settings.fitness_function = 'plp'
settings.autoscale = 10.
settings.early_termination = False
import tempfile
batch_tempd = tempfile.mkdtemp()
settings.output_directory = batch_tempd
settings.output_file = '/Applications/CCDC/Python_API_2022/docs/example_files/docked_ligands.mol2'

MLL1_ligand_file = '/Applications/CCDC/Python_API_2022/docs/example_files/SAH.mol2'
MLL1_ligand = MoleculeReader(MLL1_ligand_file)[0]
settings.add_ligand_file(MLL1_ligand_file, 10)

results = docker.dock() 

Starting GOLD with conf file /Users/williammccorkindale/ml_physics/smi2wyck/notebooks/docking/api_gold.conf
Setting up GOLD environment...
GOLD Version 2022.1.0
Running:
 
     "/Applications/CCDC/Discovery_2022/GOLD/gold/d_macx/bin/gold_macx" "/Users/williammccorkindale/ml_physics/smi2wyck/notebooks/docking/api_gold.conf"



Mpro

In [26]:
from ccdc.docking import Docker
from ccdc.io import MoleculeReader, EntryReader

docker = Docker()
settings = docker.settings

# mpro_protein_file = '/Users/williammccorkindale/Downloads/PhD_data/FRESCO/Apo_Mpro_6YB7/6YB7_model.pdb'
# settings.add_protein_file(mpro_protein_file)

mpro_dir = '/Users/williammccorkindale/Downloads/PhD_data/FRESCO/Apo_Mpro_6YB7'

# mpro_native_ligand_file = f'/Users/williammccorkindale/Downloads/PhD_data/FRESCO/Apo_Mpro_6YB7/6lu7.pdb'

mpro_protein_file = f'{mpro_dir}/6LU7_protein.mol2'
settings.add_protein_file(mpro_protein_file)

mpro_native_ligand_file = f'{mpro_dir}/6LU7_ligand.mol2'
mpro_native_ligand = MoleculeReader(mpro_native_ligand_file)[0]
print(mpro_native_ligand.smiles)
mpro_protein = settings.proteins[0]
settings.binding_site = settings.BindingSiteFromLigand(mpro_protein, mpro_native_ligand, 8.0)

settings.fitness_function = 'plp'
settings.autoscale = 10.
settings.early_termination = False

import tempfile
batch_tempd = tempfile.mkdtemp()
settings.output_directory = '/Users/williammccorkindale/ml_physics/smi2wyck/notebooks/docking/results/'
settings.output_file = 'docked_ligands.mol2'

# mpro_ligand_file = '/Applications/CCDC/Python_API_2022/docs/example_files/SAH.mol2'
mpro_ligand = MoleculeReader(mpro_native_ligand_file)[0]
settings.add_ligand_file(mpro_native_ligand_file, 5)

results = docker.dock() 
print(results.return_code)


CC(C)CC(NC(=O)C(NC(=O)C(C)NC(=O)C1=NOC(=C1)C)C(C)C)C(=O)NC(CC1CCNC1=O)[C]=CC(=O)OCc1ccccc1
Starting GOLD with conf file /Users/williammccorkindale/ml_physics/smi2wyck/notebooks/docking/api_gold.conf
Setting up GOLD environment...
GOLD Version 2022.1.0
Running:
 
     "/Applications/CCDC/Discovery_2022/GOLD/gold/d_macx/bin/gold_macx" "/Users/williammccorkindale/ml_physics/smi2wyck/notebooks/docking/api_gold.conf"

1


In [27]:
ligands = results.ligands

In [30]:
print(ligands[0].fitness())

85.4826
